In [5]:
from langchain import LLMChain
from langchain.chains import LLMChain, LLMMathChain, SequentialChain, TransformChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.tools import Tool

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("api_key")

# Set the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = api_key

In [2]:
"""Defining utility functions for constructing a readable exchange
"""

def system_output(output):
    """Function for printing out to the user"""
    print("======= Bot =======")
    print(output)


def user_input():
    """Function for getting user input"""
    print("======= Human Input =======")
    return input()


def parsing_info(output):
    """Function for printing out key info"""
    print(f"*Info* {output}")

In [9]:
class Edge:

    """Edge
    at its highest level, an edge checks if an input is good, then parses
    data out of that input if it is good
    """

    def __init__(
        self, condition, parse_prompt, parse_class, llm, max_retrys=3, out_node=None
    ):
        """
        condition (str): a True/False question about the input
        parse_query (str): what the parser whould be extracting
        parse_class (Pydantic BaseModel): the structure of the parse
        llm (LangChain LLM): the large language model being used
        """
        self.condition = condition
        self.parse_prompt = parse_prompt
        self.parse_class = parse_class
        self.llm = llm

        # how many times the edge has failed, for any reason, for deciding to skip
        # when successful this resets to 0 for posterity.
        self.num_fails = 0

        # how many retrys are acceptable
        self.max_retrys = max_retrys

        # the node the edge directs towards
        self.out_node = out_node

    def check(self, input):
        """ask the llm if the input satisfies the condition"""
        validation_query = f"following the output schema, does the input satisfy the condition?\ninput:{input}\ncondition:{self.condition}"

        class Validation(BaseModel):
            is_valid: bool = Field(description="if the condition is satisfied")

        parser = PydanticOutputParser(pydantic_object=Validation)
        input = f"Answer the user query.\n{parser.get_format_instructions()}\n{validation_query}\n"
        return parser.parse(self.llm(input)).is_valid

    def parse(self, input):
        """ask the llm to parse the parse_class, based on the parse_prompt, from the input"""
        parse_query = f'{self.parse_prompt}:\n\n"{input}"'
        parser = PydanticOutputParser(pydantic_object=self.parse_class)
        input = f"Answer the user query.\n{parser.get_format_instructions()}\n{parse_query}\n"
        return parser.parse(self.llm(input))

    def execute(self, input):
        """Executes the entire edge
        returns a dictionary:
        {
            continue: bool,       weather or not should continue to next
            result: parse_class,  the parsed result, if applicable
            num_fails: int         the number of failed attempts
        }
        """

        # input did't make it past the input condition for the edge
        if not self.check(input):
            self.num_fails += 1
            if self.num_fails >= self.max_retrys:
                return {"continue": True, "result": None, "num_fails": self.num_fails}
            return {"continue": False, "result": None, "num_fails": self.num_fails}

        try:
            # attempting to parse
            self.num_fails = 0
            return {
                "continue": True,
                "result": self.parse(input),
                "num_fails": self.num_fails,
            }
        except:
            # there was some error in parsing.
            # note, using the retry or correction parser here might be a good idea
            self.num_fails += 1
            if self.num_fails >= self.max_retrys:
                return {"continue": True, "result": None, "num_fails": self.num_fails}
            return {"continue": False, "result": None, "num_fails": self.num_fails}

In [10]:
# defining the query for the condition, and parse prompt
condition = "Does the input contain fruits?"
parse_prompt = "extract only the fruits from the following text. Do not extract any food items besides pure fruits."

In [11]:
# defining the model used in this test
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [12]:
class sampleOutputTemplate(BaseModel):
    output: str = Field(description="contact information")

In [13]:
# defining the edge
testEdge = Edge(
    condition=condition,
    parse_prompt=parse_prompt,
    parse_class=sampleOutputTemplate,
    llm=model,
)

In [14]:
sample_input = "my favorite deserts are chocolate covered strawberries, oreos, bannana splits, and cake."

In [15]:
testEdge.check(sample_input)

True

In [16]:
class Node:

    """Node
    at its highest level, a node asks a user for some input, and trys
    that input on all edges. It also manages and executes all
    the edges it contains
    """

    def __init__(self, prompt, retry_prompt):
        """
        prompt (str): what to ask the user
        retry_prompt (str): what to ask the user if all edges fail
        parse_class (Pydantic BaseModel): the structure of the parse
        llm (LangChain LLM): the large language model being used
        """

        self.prompt = prompt
        self.retry_prompt = retry_prompt
        self.edges = []

    def run_to_continue(self, _input):
        """Run all edges until one continues
        returns the result of the continuing edge, or None
        """
        for edge in self.edges:
            res = edge.execute(_input)
            if res["continue"]:
                return res
        return None

    def execute(self):
        """Handles the current conversational state
        prompots the user, tries again, runs edges, etc.
        returns the result from an adge
        """

        # initial prompt for the conversational state
        system_output(self.prompt)

        while True:
            # getting users input
            _input = user_input()

            # running through edges
            res = self.run_to_continue(_input)

            if res is not None:
                # parse successful
                parsing_info(f"parse results: {res}")
                return res

            # unsuccessful, prompting retry
            system_output(self.retry_prompt)

In [17]:
# Defining 2 edges from the node

condition1 = "Does the input contain a full and valid email?"
parse_prompt1 = "extract the email from the following text."
edge1 = Edge(condition1, parse_prompt1, sampleOutputTemplate, model)
condition2 = (
    "Does the input contain a full and valid phone number (xxx-xxx-xxxx or xxxxxxxxxx)?"
)
parse_prompt2 = "extract the phone number from the following text."
edge2 = Edge(condition2, parse_prompt2, sampleOutputTemplate, model)

In [18]:
# Defining A Node
test_node = Node(
    prompt="Please input your full email address or phone number",
    retry_prompt="I'm sorry, I didn't understand your response.\nPlease provide a full email address or phone number(in the format xxx-xxx-xxxx)",
)

In [19]:
test_node.edges = [edge1, edge2]

In [20]:
res = test_node.execute()

======= Bot =======
Please input your full email address or phone number
======= Human Input =======


In [ ]:
from data.chat import *
from data.graph import *
from data.validation import PhoneCallTicket, UserProfile, PhoneCallRequest
from graph.edge import *
from graph.chain_based_node import *
from graph.edge import BaseEdge
from graph.node import BaseNode
from graph.text_based_edge import PydanticTextBasedEdge
from tools.rag_responder import HelpCenterAgent
from tools.user_info_db import search_user_info_on_db, search_user_subscription_on_db
from tools.audio_transcribe import call_customer